In [1]:
import os, requests, time

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from urllib.parse import urlparse, urljoin
import csv
import pandas as pd

## Make lists of URLs per country

### NL

In [3]:
nl_files_list = os.listdir("../met-naam-2018/castles-netherlands/")

In [79]:
dchar_domain_extenstions = ['net','org','com','top']
nl_urls_div = [link[:-4] for link in nl_files_list if link.endswith('.txt') and link[-7:-4] in dchar_domain_extenstions]
nl_urls_div = ['https://www.'+link[:-3] + '.' + link[-3:] for link in nl_urls_div]

nl_urls_nl = ['https://www.'+item[:-6]+'.'+item.rstrip('.txt')[-2:] for item in nl_files_list]

nl_urls = nl_urls_div + nl_urls_nl

In [80]:
print(len(nl_urls), nl_urls[-1:])

58 ['https://www.artlandt.op']


In [82]:
nl_urls[10:20]

['https://www.slotzuylen.nl',
 'https://www.kasteelgeldrop.nl',
 'https://www.fraeylemaborg.nl',
 'https://www.slotloevestein.nl',
 'https://www.museumijsselstein.nl',
 'https://www.twickel.nl',
 'https://www.kasteelduivenvoorde.nl',
 'https://www.huystendonck.nl',
 'https://www.kasteeldussen.nl',
 'https://www.huisbergh.nl']

In [19]:
print(len(nl_urls), nl_urls[-1:])

58 ['https://www.artlandt.op']


In [21]:
# optional: save this list as a csv file
df_nl = pd.DataFrame(data={"NL domain": nl_urls})
df_nl.to_csv("../url_lists/nl_urls.csv", sep=',',index=False)

### FR

In [179]:
fr_files_list = os.listdir("../met-naam-2018/castles-france/")
len(fr_files_list)

92

In [180]:
domain_extensions = ['com', 'net', 'org', 'bzh']
fr_urls_div = [link[:-4] for link in fr_files_list if link.endswith('.txt') and link[-7:-4] in domain_extensions]
fr_urls_div = ['http://www.'+link[:-3] + '.' + link[-3:] for link in fr_urls_div]


fr_urls_fr = [link[:-4] for link in fr_files_list if link.endswith('.txt') and link[-6:-4]== 'fr']
fr_urls_fr = ['http://www.'+link[:-2] + '.' + link[-2:] for link in fr_urls_fr]

#catch the unusual .corsica domains
fr_urls_corsica = [link[:-4] for link in fr_files_list if link.endswith('.txt') and link[-11:-4]== 'corsica']
fr_urls_corsica = ['http://'+link[:-7] + '.' + link[-7:] for link in fr_urls_corsica]

fr_urls = fr_urls_div + fr_urls_fr + fr_urls_corsica
print(len(fr_urls))

91


In [181]:
# optional: save this list as a csv file
df_fr = pd.DataFrame(data={"FR domain": fr_urls})
df_fr.to_csv("../url_lists/fr_urls.csv", sep=',',index=False)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    print (df_fr)

### DE

In [185]:
de_files_list = os.listdir("../met-naam-2018/castles-germany/")
len(de_files_list)

90

In [186]:
dchar_domain_extenstions = ['net','org','com']
de_urls_div = [link[:-4] for link in de_files_list if link.endswith('.txt') and link[-7:-4] in dchar_domain_extenstions]
de_urls_div = ['http://www.'+link[:-3] + '.' + link[-3:] for link in de_urls_div]

cchar_domain_extensions = ['de','nl','eu']
de_urls_de = [link[:-4] for link in de_files_list if link.endswith('.txt') and link[-6:-4] in cchar_domain_extensions]
de_urls_de = ['http://www.'+link[:-2] + '.' + link[-2:] for link in de_urls_de]

#catch the .info domain
de_urls_info = [link[:-4] for link in de_files_list if link.endswith('.txt') and link[-8:-4]== 'info']
de_urls_info = ['http://'+link[:-4] + '.' + link[-4:] for link in de_urls_info]

de_urls = de_urls_div + de_urls_de + de_urls_info
print(len(de_urls))

90


In [191]:
# optional: save this list as a csv file
df_de = pd.DataFrame(data={"DE domain": de_urls})
df_de.to_csv("../de_urls.csv", sep=',',index=False)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    print (df_de)

### UK

In [229]:
uk_files_list = os.listdir("../met-naam-2018/castles-united-kingdom/")
len(uk_files_list)

76

In [230]:
dchar_domain_extenstions = ['net','org','com']
uk_urls_div = [link[:-4] for link in uk_files_list if link.endswith('.txt') and link[-7:-4] in dchar_domain_extenstions]
uk_urls_div = ['https://www.'+link[:-3] + '.' + link[-3:] for link in uk_urls_div]

cchar_domain_extensions_5 = ['govuk','orguk']
uk_urls_gov = [link[:-4] for link in uk_files_list if link.endswith('.txt') and link[-9:-4] in cchar_domain_extensions_5]
uk_urls_gov = ['https://www.'+link[:-5] + '.' + link[-5:-2] + '.' + link[-2:] for link in uk_urls_gov]

cchar_domain_extensions_4 = ['couk','acuk']
uk_urls_co = [link[:-4] for link in uk_files_list if link.endswith('.txt') and link[-8:-4] in cchar_domain_extensions_4]
uk_urls_co = ['https://www.'+link[:-4] + '.' + link[-4:-2] + '.' + link[-2:] for link in uk_urls_co]

uk_urls = uk_urls_div + uk_urls_gov + uk_urls_co + ['https://www.hemyockcastle.uk']
print(len(uk_urls))

76


In [232]:
# optional: save this list as a csv file
df_uk = pd.DataFrame(data={"UK domain": uk_urls})
df_uk.to_csv("../uk_urls.csv", sep=',',index=False)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    print (df_uk)

## URL Check up functions 

In [5]:
def check_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            print(0)
        else:
            print(f"The URL '{url}' is unreachable. Status code: {response.status_code}")
    except requests.ConnectionError:
        print(f"The URL '{url}' is unreachable.")

In [6]:
cc = 'FR'

In [8]:
# optional: change between existing url list (made in this notebook) and a locally stored csv list 
domain = pd.read_csv('../url_lists/'+cc+'_urls.csv')
urls = domain['FR domains'].values.tolist() 

for i in range(len(urls)):
    #print(urls[i])
    check_url(urls[i])

The URL 'https://www.la-hunaudaye.com/' is unreachable. Status code: 403
0
0
The URL 'https://www.chateau-eaucourt.com' is unreachable. Status code: 429
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
The URL 'https://www.chateau-pirou.fr/' is unreachable. Status code: 410
0
0
0
0
0
0
0
0
0
0
The URL 'https://www.chateau-picquigny.com' is unreachable. Status code: 429
The URL 'https://www.chateau-bazoches.com' is unreachable.
0
0
0
0
0
0
0
The URL 'https://www.chateau-malbrouck.com' is unreachable.
0
0
0
The URL 'https://www.chateau-de-la-rochepot.com' is unreachable.
0
0
0
0
0
0
0
0
0
The URL 'https://www.chateaugavray.fr' is unreachable. Status code: 555
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
The URL 'https://www.chateaudetanlay.fr' is unreachable. Status code: 429
The URL 'https://www.folleville-chateau-medieval.fr' is unreachable. Status code: 429
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


## Notes on URLS: 

#### DUTCH CASTLES (XX/55)
- https://www.museumdefundatie.nl/ is not on the 2018 list
- https://www.beeldentuinravesteyn.nl changed domain to: https://www.ruine-ravesteyn.nl/

#### FRENCH CASTLES (6/91)
doesn't respond to checkup function (errors 429, 403 and 555) but the link works - it is correct: 
- https://www.chateau-eaucourt.com/ 
- http://www.chateau-picquigny.com
- https://www.la-hunaudaye.com/
- https://www.chateaudetanlay.fr/
- https://www.chateaugavray.fr/

seems to be closed, link won't open but WHOIS listing exists:
- www.chateau-de-la-rochepot.com (wikipedia: https://en.wikipedia.org/wiki/Ch%C3%A2teau_de_la_Rochepot "In October 2018, the castle was seized by the French government after investigation into an alleged money laundering scheme by Dmytro Malynovskyi, a Ukrainian. Malynovski was arrested after an investigation into a corruption and money laundering scheme in which he purchased and lived in the castle after faking his own death.")
 
#### GERMAN CASTLES (10/90)
doesn't respond to checkup function (errors 429 and 403) but the link is correct:
- https://www.schlossparkmarisfeld.de/
- https://www.schloss-marienburg.de/
- https://schloss-langenburg.de/
- https://schloss-karlsberg.de/ (secure connection missing or required?)
- http://www.schloss-braunfels.de'

possible mistakes and changes to domain names:
- wwwkasteelbentheimnl -> https://burg-bentheim.de/ text is in dutch in German 2018 scraped material
- http://evenburg.landkreis-leer.de/ redirects to http://landkreis-leer.de/ the castle does not have its own domain since 2022, review the subdomain  wmk.landkreis-leer.de/ as replacement castle representation
- http://www.wilhelmshoehede.txt requires Flash (very outdated site, no accessible content)
- https://www.schloesser-hessen.de/de/schloss-erbach is a collection of castles? thus https://www.schloss-erbach.de redirects here
- http://burghohnstein.com is for sale, it is replaces by http://burghohnstein.info which is included in the list here

#### UK CASTLES (12/76)
doesn't respont to checkup function (error 429, 403):
- https://www.hrp.org.uk/ - is a collection of palaces - perhaps there are individual websites?
- https://www.hants.gov.uk/
- https://www.haddonhall.co.uk/
- https://www.tamworthcastle.co.uk/
- https://www.whittingtoncastle.co.uk only reachable without 'www' (as https://whittingtoncastle.co.uk/)

possible mistake:
- why is https://dur.ac.uk/ on the list? it is the university website, no (direct) castle content

closed:
- https://wresslecastle.org/ domain not maintained (people speak of using Facebook page to inquire about opening hours), Selena removed from url list Nov 2024
- https://www.leicestercastle.co.uk domain not maintained, information on the castle available on history of Leicestre website, https://www.storyofleicester.info/

changed domains:
- http://www.rabycastle.com is now https://www.raby.co.uk 
- http://www.castlerising.com is now http://www.castlerising.co.uk
- http://www.fobt.org.uk is now https://friendsofbuckdentowers.co.uk/
- https://www.turtontower.co.uk is now https://turtontower.com/




## Notes for writing
- visit https://castlestudiestrust.org/ and understand what they mean by 'cutting edge castle research'
- narrate how domains differ for countries - NL are all .nl except for 2 .com, while FR, DE and UK have different endings; this incoherency is reflected in the different character of top-level domain endings - some FR, DE and UK casltes have the commercial .com and co.uk ending, while others go with .org and .net which are in principle for non-profit organisations; .de and .fr (and 1 .uk) domains reflect the standard country association, while only some UK castles have gov.uk castles, directly related to the government
- General questions:
1. what is a web presentation of the castle, what cultural meaning does it cary, how is digital communication important to castle preservation and management?
 

# Scrape function

In [46]:
def scrape_website(url, visited_urls=set()):
    try:
        if url in visited_urls:
            return ""
        
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            visited_urls.add(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            content = soup.get_text()
            
            # Find all internal links and recursively scrape them
            internal_links = soup.find_all('a', href=True)
            for link in internal_links:
                absolute_url = urljoin(url, link['href'])
                if urlparse(absolute_url).netloc == urlparse(url).netloc:
                    content += scrape_website(absolute_url, visited_urls)
            
            return content
        elif response.status_code == 403:
            print(f"403 Forbidden: {url}")
            return ""
        else:
            print(f"Error scraping {url}: {response.status_code}")
            return ""
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return ""

def write_to_file(content, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)

In [71]:
# Test scrape one website
write_path="../TEST/"
url = nl_urls[44]

def main():
    domain = urlparse(url).netloc
    filename = write_path+f"{domain}.txt"
    write_to_file("STARTING", filename)
    content = scrape_website(url)
    #print(content)
    if content:
        print('there is', content)
        write_to_file(content, filename)
        print(f"Scraped {url} and wrote content to {filename}")
    else:
        print(f"Failed to scrape {url}")
        
if __name__ == "__main__":
    main()

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
# Get all Dutch websites or a range of urls list
write_path="../met-naam-2024/castles-netherlands-1/"

def main():
    for url in nl_urls[0:6]:
        try:
            domain = urlparse(url).netloc
            filename = write_path+f"{domain}.txt"
            content = scrape_website(url)
            if content:
                write_to_file(content, filename)
                print(f"Scraped {url} and wrote content to {filename}")
            else:
                print(f"Failed to scrape {url}")
            # Introduce a delay between requests to avoid overwhelming the server
            time.sleep(2)
        except Exception as e:
            print(f"Error processing {url}: {e}")

if __name__ == "__main__":
    main()

#### Test one url in Wayback machine

In [12]:
url= "https://web.archive.org/web/20180511012246/https://www.dekemastate.nl/"

In [13]:
content = scrape_website(url)
filename = 'wwwdekemastatenl.txt'
if content:
    write_to_file(content, filename)
    print(f"Scraped {url} and wrote content to {filename}")
else:
    print(f"Failed to scrape {url}")

Failed to scrape https://web.archive.org/web/20180511012246/https://www.dekemastate.nl/


In [107]:
soup1 = BeautifulSoup(response.text)

In [108]:
allowed_tags = ['div','p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'span', 'br']
all_text = ' '.join([element.get_text() for element in soup.find_all(allowed_tags)])
with open(url[29:33], "w") as file:
    file.write(all_text)

## CommonCrawl test

In [1]:
import requests
import json
import os
import pandas as pd
# For parsing URLs:
from urllib.parse import quote_plus
from bs4 import BeautifulSoup

In [2]:
def search_cc_index(url, index_name):
    """
    Search the Common Crawl Index for a given URL.

    This function queries the Common Crawl Index API to find records related to the specified URL. 
    It uses the index specified by `index_name` to retrieve the data and returns a list of JSON objects, 
    each representing a record from the index.

    Arguments:
        url (str): The URL to search for in the Common Crawl Index.
        index_name (str): The name of the Common Crawl Index to search (e.g., "CC-MAIN-2024-10").

    Returns:
        list: A list of JSON objects representing records found in the Common Crawl Index. 
              Returns None if the request fails or no records are found.

    Example:
        >>> search_cc_index("example.com", "CC-MAIN-2024-10")
        [{...}, {...}, ...]
    """
    encoded_url = quote_plus(url)
    index_url = f'http://index.commoncrawl.org/{index_name}-index?url={encoded_url}&output=json'
    response = requests.get(index_url)

    if response.status_code == 200:
        records = response.text.strip().split('\n')
        return [json.loads(record) for record in records]
    else:
        return None



def fetch_single_record(warc_record_filename, offset, length):
    """
    Fetch a single WARC record from Common Crawl.

    Arguments:
        record {dict} -- A dictionary containing the WARC record details.

    Returns:
        bytes or None -- The raw content of the response if found, otherwise None.
    """
    
    s3_url = f'https://data.commoncrawl.org/{warc_record_filename}'

    # Define the byte range for the request
    byte_range = f'bytes={offset}-{offset + length - 1}'

    # Send the HTTP GET request to the S3 URL with the specified byte range
    response = requests.get(
        s3_url,
        headers={'Range': byte_range},
        stream=True
    )

    if response.status_code == 206:
        # Use `stream=True` in the call to `requests.get()` to get a raw byte stream,
        # because it's gzip compressed data
        stream = ArchiveIterator(response.raw)
        for warc_record in stream:
            if warc_record.rec_type == 'response':
                return warc_record.content_stream().read()
    else:
        print(f"Failed to fetch data: {response.status_code}")
    
    return None


def append_df_row_to_pickle(row, pickle_file):
    """
    Append a row to a DataFrame stored in a pickle file.
    
    Arguments:
        row {pd.Series} -- The row to be appended to the DataFrame.
        pickle_file {str} -- The path to the pickle file where the DataFrame is stored.
    """
    # Check if the pickle file exists
    if os.path.exists(pickle_file):
        # Load the existing DataFrame from the pickle file
        df = pd.read_pickle(pickle_file)
    else:
        # If the file doesn't exist, create a new DataFrame
        df = pd.DataFrame(columns=row.index)

    # Append the new row to the DataFrame
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    
    # Save the updated DataFrame back to the pickle file
    df.to_pickle(pickle_file)


def load_processed_indices(pickle_file):
    """
    Load processed indices from a pickle file to check previously processed records.

    Arguments:
        pickle_file {str} -- The path to the pickle file where the DataFrame is stored.
    
    Returns:
        Set of processed indices.
    """
    if os.path.exists(pickle_file):
        df = pd.read_pickle(pickle_file)
        # Assuming 'index' column is in the DataFrame and contains indices of processed records
        processed_indices = set(df['index'].unique())
        print(f"Loaded {len(processed_indices)} processed indices from {pickle_file}")
        return processed_indices
    else:
        print(f"No processed indices found. Pickle file '{pickle_file}' does not exist.")
        return set()

In [15]:
# The URL you want to look up in the Common Crawl index
target_url = 'https://www.slotzuylen.nl/*'  # Replace with your target URL

# list of indexes https://commoncrawl.org/get-started
indexes  = ['CC-MAIN-2024-33','CC-MAIN-2024-30','CC-MAIN-2024-26']

record_dfs = []

# Fetch each index and store into a dataframe
for index_name in indexes:
    print('Running: ', index_name)
    records = search_cc_index(target_url,index_name)
    record_df = pd.DataFrame(records)
    record_df['index_name'] = index_name
    record_dfs.append(record_df)

# Combine individual dataframes
all_records_df = pd.concat(record_dfs)
all_records_df = all_records_df.sort_values(by='index_name', ascending=False)
all_records_df = all_records_df.reset_index()

# Create columns where to store data later
all_records_df['success_status'] = 'not processed'
all_records_df['html'] = ''

Running:  CC-MAIN-2024-33
Running:  CC-MAIN-2024-30
Running:  CC-MAIN-2024-26


In [16]:
all_records_df.to_csv('all_records_df.csv', index=False)

In [17]:
df = all_records_df

In [18]:
# safeguareds
# If pickle file exists, check for processed items
pickle_file = 'commcrawl_indeed.pkl'
processed_indices = load_processed_indices(pickle_file)
#if processed_indices:
#    # Remove processed items
#    df = df[~df['index'].isin(processed_indices)]

# Create storage for later
successful = set()
results = {}

# Keep track of each row processed
i = 0 
perc = 0
n_records = len(df)
print(f"Found {n_records} records for {target_url}")
mod = int(n_records * 0.01)

Loaded 100 processed indices from commcrawl_indeed.pkl
Found 235 records for https://www.slotzuylen.nl/*


In [19]:
from warcio.archiveiterator import ArchiveIterator

# Reset index to help with looping
df.reset_index(drop=True,inplace=True)


for i in range(len(df)):
    # Print every 1% process
    if i % mod == 0: 
        print(f'{i} of {n_records}: {perc}%')
        perc += 1

    record_url = df.loc[i, 'url']

    # Fetch only URLs that were not processed
    # If it was already processed, skip URL 
    # (Helps speeding if you only need one version of the HTML, not its history)
    if not record_url in successful:
        length = int(df.loc[i, 'length'])
        offset = int(df.loc[i, 'offset'])
        warc_record_filename = df.loc[i, 'filename']
        result = fetch_single_record(warc_record_filename, offset, length)
        
        if not result:
            df.loc[i,'success_status'] = 'invalid warc'
        else:
            df.loc[i,'success_status'] = 'success'
            df.loc[i,'html'] = result
    else: 
        df.loc[i,'success_status'] = 'previously processed'

    # Add to pickle file
    append_df_row_to_pickle(df.loc[i, :], pickle_file)

0 of 235: 0%
2 of 235: 1%
4 of 235: 2%
6 of 235: 3%
8 of 235: 4%
10 of 235: 5%
12 of 235: 6%
14 of 235: 7%
16 of 235: 8%
18 of 235: 9%
20 of 235: 10%
22 of 235: 11%
24 of 235: 12%
26 of 235: 13%
28 of 235: 14%
30 of 235: 15%
32 of 235: 16%
34 of 235: 17%
36 of 235: 18%
38 of 235: 19%
40 of 235: 20%
42 of 235: 21%
44 of 235: 22%
46 of 235: 23%
48 of 235: 24%
50 of 235: 25%
52 of 235: 26%
54 of 235: 27%
56 of 235: 28%
58 of 235: 29%
60 of 235: 30%
62 of 235: 31%
64 of 235: 32%
66 of 235: 33%
68 of 235: 34%
70 of 235: 35%
72 of 235: 36%
74 of 235: 37%
76 of 235: 38%
78 of 235: 39%
80 of 235: 40%
82 of 235: 41%
Failed to fetch data: 503
84 of 235: 42%
86 of 235: 43%
Failed to fetch data: 503
88 of 235: 44%
90 of 235: 45%
Failed to fetch data: 503
92 of 235: 46%
94 of 235: 47%
96 of 235: 48%
98 of 235: 49%
100 of 235: 50%
102 of 235: 51%
104 of 235: 52%
106 of 235: 53%
108 of 235: 54%
110 of 235: 55%
112 of 235: 56%
114 of 235: 57%
116 of 235: 58%
118 of 235: 59%
120 of 235: 60%
122 of 235:

In [20]:
commoncrawl_data = pd.read_pickle(pickle_file)
commoncrawl_data[
    ['url','filename','index_name','success_status','html']
    ].head()

,url,filename,index_name,success_status,html
0,https://slotzuylen.nl/,crawl-data/CC-MAIN-2024-33/segments/1722640427...,CC-MAIN-2024-33,success,"b' <!doctype html>\r\n<html lang=""nl-NL"">\r..."
1,https://slotzuylen.nl/agenda/overzicht/?relate...,crawl-data/CC-MAIN-2024-33/segments/1722640427...,CC-MAIN-2024-33,success,"b' <!doctype html>\r\n<html lang=""nl-NL"">\r..."
2,https://slotzuylen.nl/bewoners/,crawl-data/CC-MAIN-2024-33/segments/1722640427...,CC-MAIN-2024-33,success,"b' <!doctype html>\r\n<html lang=""nl-NL"">\r..."
3,https://slotzuylen.nl/agenda/zomeravondrondlei...,crawl-data/CC-MAIN-2024-33/segments/1722640427...,CC-MAIN-2024-33,success,"b' <!doctype html>\r\n<html lang=""nl-NL"">\r..."
4,https://slotzuylen.nl/agenda/zomeravondrondlei...,crawl-data/CC-MAIN-2024-33/segments/1722640427...,CC-MAIN-2024-33,success,"b' <!doctype html>\r\n<html lang=""nl-NL"">\r..."


In [21]:
from  bs4 import BeautifulSoup

# Select HTML from row 0
content = commoncrawl_data.loc[0, 'html']

# Parse in Beautiful soup
soup = BeautifulSoup(content, 'html.parser')
print(soup.find('title'))

<title>Kasteel - Slot Zuylen</title>
